In [ ]:
# Install AutoGluon
!pip install -q autogluon

import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor

In [ ]:
# Load data
train_path = '/kaggle/input/super-ai-engineer-5-internet-of-things-attack/IOT/train.csv'
test_path = '/kaggle/input/super-ai-engineer-5-internet-of-things-attack/IOT/test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Preprocessing
target = 'attack_type'

# Drop 'id' column from test_df (it's not a feature)
test_ids = test_df['id']  # Save the 'id' column for submission
test_df = test_df.drop(columns=['id'])

# Define hyperparameters for AutoGluon
hyperparameters = {
    'GBM': [
        {'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
        {},
        'GBMLarge',
    ],
    'CAT': {},  # CatBoost
    'XGB': {},  # XGBoost
    'NN_TORCH': {  # Neural Network
        'num_epochs': 10,
        'learning_rate': 1e-3,
    },
    'RF': {  # Random Forest
        'criterion': 'gini',
        'max_depth': 15,
    },
}

# Initialize AutoGluon TabularPredictor with advanced settings
predictor = TabularPredictor(
    label=target,
    eval_metric='accuracy',
).fit(
    train_data=train_df,
    hyperparameters=hyperparameters,
    time_limit=3600,  # Increase time limit to 1 hour
    num_bag_folds=5,  # Use 5-fold bagging
    num_stack_levels=2,  # Use 2 levels of stacking
    hyperparameter_tune_kwargs={  # Enable hyperparameter tuning
        'scheduler': 'local',
        'searcher': 'auto',
    },
    verbosity=2,  # Increase verbosity for more detailed logs
)

# Make predictions on the test set
predicted_attack_types = predictor.predict(test_df)

# Create submission DataFrame
submission_df = pd.DataFrame({
    'id': test_ids,  # Use the saved 'id' column from the test set
    'attack_type': predicted_attack_types
})

# Save submission file
submission_df.to_csv('/kaggle/working/submissionIot_attack_autogluon_enhanced.csv', index=False)

print("Submission file created successfully!")